In [31]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd

In [32]:
train = pd.read_csv("C:/Users/Aravind/PROJECTS/Used-Car-Price-Prediction/Data/train.csv")
test = pd.read_csv("C:/Users/Aravind/PROJECTS/Used-Car-Price-Prediction/Data/test.csv")

In [33]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            54273 non-null  int64 
 1   brand         54273 non-null  object
 2   model         54273 non-null  object
 3   model_year    54273 non-null  int64 
 4   milage        54273 non-null  int64 
 5   fuel_type     54273 non-null  object
 6   engine        54273 non-null  object
 7   transmission  54273 non-null  object
 8   ext_col       54273 non-null  object
 9   int_col       54273 non-null  object
 10  accident      54273 non-null  object
 11  clean_title   54273 non-null  object
 12  price         54273 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 5.4+ MB


In [34]:
train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


In [35]:
for column in train.select_dtypes(include=['object']).columns:
    print(f"Value counts for column '{column}':")
    print(train[column].value_counts())
    print()

Value counts for column 'brand':
brand
BMW              7369
Ford             6706
Mercedes-Benz    5087
Chevrolet        4424
Audi             2922
Porsche          2627
Toyota           2315
Lexus            2257
Jeep             2247
Land             1995
Cadillac         1565
Nissan           1252
GMC              1076
Dodge            1011
RAM               966
INFINITI          957
Lincoln           767
Mazda             748
Subaru            739
Hyundai           694
Jaguar            661
Volkswagen        628
Honda             624
Acura             580
Kia               526
Volvo             452
MINI              364
Maserati          293
Bentley           284
Chrysler          258
Genesis           249
Buick             228
Mitsubishi        182
Hummer            176
Pontiac           149
Alfa              144
Rolls-Royce       142
Lamborghini       122
Tesla             110
Ferrari            87
Saturn             58
Scion              53
Aston              50
McLaren        

In [36]:
import re

def parse_transmission(transmission_series):
    def parse_entry(entry):
        # Handle non-string entries
        if not isinstance(entry, str):
            return {
                'Speed': None,
                'Type': None,
                'Features': None,
            }
        
        # Initialize dictionary for parsed values
        parsed = {
            'Speed': None,
            'Type': None,
            'Features': None,
        }
        
        # Regex patterns
        speed_pattern = re.compile(r'(\d+)-Speed')
        type_pattern = re.compile(r'(A/T|M/T|Automatic|Manual|CVT|DCT)')
        features_pattern = re.compile(r'with (.+)|\((.+)\)|w/(.+)')
        
        # Find speed
        speed_match = speed_pattern.search(entry)
        if speed_match:
            parsed['Speed'] = int(speed_match.group(1))
        
        # Find type
        type_match = type_pattern.search(entry)
        if type_match:
            parsed['Type'] = type_match.group()
        else:
            # Attempt to categorize if type is not explicit
            if 'Automatic' in entry:
                parsed['Type'] = 'Automatic'
            elif 'Manual' in entry:
                parsed['Type'] = 'Manual'
            elif 'CVT' in entry:
                parsed['Type'] = 'CVT'
        
        # Find features
        features_match = features_pattern.search(entry)
        if features_match:
            features = features_match.groups()
            parsed['Features'] = ', '.join(filter(None, features))
        else:
            # Check for common features not captured by the regex
            if 'Dual Shift Mode' in entry:
                parsed['Features'] = 'Dual Shift Mode'
            elif 'Overdrive' in entry:
                parsed['Features'] = 'Overdrive'
            elif 'Auto-Shift' in entry:
                parsed['Features'] = 'Auto-Shift'
        
        return parsed

    # Apply the parsing function to each entry
    parsed_data = transmission_series.apply(parse_entry)
    
    # Convert the list of dictionaries to a DataFrame
    parsed_df = pd.DataFrame(parsed_data.tolist())
    
    # Return the parsed DataFrame
    return parsed_df

In [37]:
transmission_series = train['transmission']

In [38]:
train_parsed = parse_transmission(transmission_series)

In [39]:
transmission_series = test['transmission']
test_parsed = parse_transmission(transmission_series)

In [40]:
train_parsed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Speed     29406 non-null  float64
 1   Type      47741 non-null  object 
 2   Features  6703 non-null   object 
dtypes: float64(1), object(2)
memory usage: 1.2+ MB


In [41]:
train_parsed.head()

,Speed,Type,Features
0,10.0,A/T,None
1,6.0,M/T,None
2,6.0,A/T,None
3,NaN,None,Dual Shift Mode
4,NaN,A/T,None


In [42]:
train = pd.concat([train, train_parsed], axis=1)
test = pd.concat([test , test_parsed],axis=1)

In [43]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            54273 non-null  int64  
 1   brand         54273 non-null  object 
 2   model         54273 non-null  object 
 3   model_year    54273 non-null  int64  
 4   milage        54273 non-null  int64  
 5   fuel_type     54273 non-null  object 
 6   engine        54273 non-null  object 
 7   transmission  54273 non-null  object 
 8   ext_col       54273 non-null  object 
 9   int_col       54273 non-null  object 
 10  accident      54273 non-null  object 
 11  clean_title   54273 non-null  object 
 12  price         54273 non-null  int64  
 13  Speed         29406 non-null  float64
 14  Type          47741 non-null  object 
 15  Features      6703 non-null   object 
dtypes: float64(1), int64(4), object(11)
memory usage: 6.6+ MB


In [44]:
train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,Speed,Type,Features
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000,10.0,A/T,None
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250,6.0,M/T,None
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000,6.0,A/T,None
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500,NaN,None,Dual Shift Mode
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850,NaN,A/T,None


In [45]:
train = train.drop(['id','clean_title'],axis=1)
test = test.drop(['clean_title'],axis=1)

In [46]:
def parse_engine(engine_str):
    parts = engine_str.split()
    horsepower, displacement, cylinder, engine_type, fuel = (np.nan, np.nan, np.nan, np.nan, np.nan)
    
    try:
        # Parse horsepower
        if 'HP' in parts[0]:
            horsepower = parts[0].replace('HP', '')
            parts.pop(0)
        
        # Parse displacement
        if 'L' in parts[0]:
            displacement = parts[0].replace('L', '')
            parts.pop(0)
        elif 'Liter' in parts[0]:
            displacement = parts[0].replace('Liter', '')
            parts.pop(0)

        # Parse cylinder
        if parts[1] == 'Cylinder':
            cylinder = parts[0]
            parts.pop(0)
            parts.pop(0)
        
        # Parse engine type and fuel
        if len(parts) >= 2:
            engine_type = parts[0]
            fuel = parts[-2] if 'Fuel' in parts[-1] else parts[-1]
    except:
        pass
    
    return pd.Series([horsepower, displacement, cylinder, engine_type, fuel])

# Apply the function to the 'engine' column and create new columns
train[['horsepower', 'displacement', 'cylinder', 'type', 'fuel']] = train['engine'].apply(parse_engine)
test[['horsepower', 'displacement', 'cylinder', 'type', 'fuel']] = test['engine'].apply(parse_engine)

In [47]:
train.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,price,Speed,Type,Features,horsepower,displacement,cylinder,type,fuel
0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,11000,10.0,A/T,None,375.0,3.5,V6,Engine,Gasoline
1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,8250,6.0,M/T,None,300.0,3.0,NaN,Straight,Gasoline
2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,15000,6.0,A/T,None,300.0,4.2,8,Engine,Gasoline
3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,63500,NaN,None,Dual Shift Mode,335.0,3.0,NaN,Straight,Hybrid
4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,7850,NaN,A/T,None,200.0,3.8,V6,Engine,Gasoline


In [48]:
train = train.drop(['model','engine','transmission','ext_col','int_col','fuel','Features'],axis=1)
test = test.drop(['model','engine','transmission','ext_col','int_col','fuel','Features'],axis=1)

In [49]:
train.head()

,brand,model_year,milage,fuel_type,accident,price,Speed,Type,Features,horsepower,displacement,cylinder,type
0,Ford,2018,74349,Gasoline,None reported,11000,10.0,A/T,None,375.0,3.5,V6,Engine
1,BMW,2007,80000,Gasoline,None reported,8250,6.0,M/T,None,300.0,3.0,NaN,Straight
2,Jaguar,2009,91491,Gasoline,None reported,15000,6.0,A/T,None,300.0,4.2,8,Engine
3,BMW,2022,2437,Hybrid,None reported,63500,NaN,None,Dual Shift Mode,335.0,3.0,NaN,Straight
4,Pontiac,2001,111000,Gasoline,None reported,7850,NaN,A/T,None,200.0,3.8,V6,Engine


In [51]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

def preprocess_data(df):
    # Selecting numerical columns
    numerical_columns = df.select_dtypes(include=['number']).columns
    df_numerical = df[numerical_columns]
    
    # Handling missing values for numerical columns using SimpleImputer with mean strategy
    imputer_numerical = SimpleImputer(strategy='mean')
    df_imputed_numerical = pd.DataFrame(imputer_numerical.fit_transform(df_numerical), columns=df_numerical.columns)
    
    # Selecting categorical columns
    categorical_columns = df.select_dtypes(include=['object']).columns
    df_categorical = df[categorical_columns]
    
    # Handling missing values for categorical columns using SimpleImputer with most frequent strategy
    imputer_categorical = SimpleImputer(strategy='most_frequent')
    df_imputed_categorical = pd.DataFrame(imputer_categorical.fit_transform(df_categorical), columns=df_categorical.columns)
    
    # Encoding categorical variables using OrdinalEncoder
    ordinal_encoder = OrdinalEncoder()
    df_encoded = pd.DataFrame(ordinal_encoder.fit_transform(df_imputed_categorical), columns=df_imputed_categorical.columns)
    
    # Merging with non-categorical columns
    df_processed = pd.concat([df_imputed_numerical, df_encoded], axis=1)
    
    return df_processed

# Assuming df is your DataFrame
# Replace 'df' with the name of your DataFrame
train_processed = preprocess_data(train)
test_processed = preprocess_data(test)


In [53]:
train_processed.head()

,model_year,milage,price,Speed,brand,fuel_type,accident,Type,Features,horsepower,displacement,cylinder,type
0,2018.0,74349.0,11000.0,10.000000,14.0,2.0,1.0,0.0,4.0,203.0,21.0,6.0,15.0
1,2007.0,80000.0,8250.0,6.000000,4.0,2.0,1.0,4.0,4.0,152.0,17.0,5.0,25.0
2,2009.0,91491.0,15000.0,6.000000,21.0,2.0,1.0,0.0,4.0,152.0,27.0,5.0,15.0
3,2022.0,2437.0,63500.0,7.107155,4.0,3.0,1.0,6.0,1.0,181.0,17.0,5.0,25.0
4,2001.0,111000.0,7850.0,7.107155,40.0,2.0,1.0,0.0,4.0,73.0,24.0,6.0,15.0


In [54]:
from scipy import stats

# Calculate the Z-scores of each column
z_scores = np.abs(stats.zscore(train_processed))

# Set a threshold for identifying outliers
threshold = 3

# Identify the indices of rows containing outliers
outliers = np.where(z_scores > threshold)
outlier_indices = set(outliers[0])

# Drop the rows with outliers
train_processed = train_processed.drop(outlier_indices).reset_index(drop=True)

In [55]:
train_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50035 entries, 0 to 50034
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model_year    50035 non-null  float64
 1   milage        50035 non-null  float64
 2   price         50035 non-null  float64
 3   Speed         50035 non-null  float64
 4   brand         50035 non-null  float64
 5   fuel_type     50035 non-null  float64
 6   accident      50035 non-null  float64
 7   Type          50035 non-null  float64
 8   Features      50035 non-null  float64
 9   horsepower    50035 non-null  float64
 10  displacement  50035 non-null  float64
 11  cylinder      50035 non-null  float64
 12  type          50035 non-null  float64
dtypes: float64(13)
memory usage: 5.0 MB


In [57]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error , r2_score
import xgboost as xgb

# Split the data into features and target variable
X = train_processed.drop(columns='price')  # Features
y = train_processed['price']  # Target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost model
xgb_reg = xgb.XGBRegressor(random_state=42)

# Train the model
xgb_reg.fit(X_train, y_train)

# Make predictions
y_pred = xgb_reg.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

RMSE: 21724.62008409035


In [58]:
import statsmodels.api as sm

X = train_processed.drop(columns='price') 
y = train_processed['price']

X = sm.add_constant(X)

X_train , X_test , y_train , y_test = train_test_split(X,y,random_state=42)

reg = sm.OLS(y_train , X_train).fit()

reg.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Ordinary least squares
=================================================================================
Model:                   OLS                  Adj. R-squared:         0.405      
Dependent Variable:      price                AIC:                    859141.4348
Date:                    2024-06-12 15:26     BIC:                    859252.3611
No. Observations:        37526                Log-Likelihood:         -4.2956e+05
Df Model:                12                   F-statistic:            2133.      
Df Residuals:            37513                Prob (F-statistic):     0.00       
R-squared:               0.406                Scale:                  5.1328e+08 
---------------------------------------------------------------------------------
                 Coef.      Std.Err.     t     P>|t|      [0.025        0.975]   
---------------------------------------------------------------------------------
const        -2109847.1731 65045.8040 -32.4363 0.0000 -2237338.7197 -1982355.6264
model_year       1052.3072    32.2616  32.6180 0.0000      989.0736     1115.5407
milage             -0.1697     0.0034 -49.2746 0.0000       -0.1765       -0.1630
Speed             989.3804   130.4672   7.5834 0.0000      733.6611     1245.0996
brand             123.6569     8.2653  14.9610 0.0000      107.4567      139.8571
fuel_type        2558.0273   482.7750   5.2986 0.0000     1611.7752     3504.2795
accident         3107.6365   277.0326  11.2176 0.0000     2564.6452     3650.6279
Type              150.9767   108.6349   1.3898 0.1646      -61.9507      363.9040
Features           33.1525   230.9579   0.1435 0.8859     -419.5313      485.8363
horsepower        132.3263     2.8839  45.8841 0.0000      126.6737      137.9789
displacement      -12.0367    15.8049  -0.7616 0.4463      -43.0148       18.9414
cylinder        -1104.3612   128.9716  -8.5628 0.0000    -1357.1491     -851.5733
type              163.1862    35.5018   4.5966 0.0000       93.6016      232.7708
---------------------------------------------------------------------------------
Omnibus:                 29871.581          Durbin-Watson:             2.003     
Prob(Omnibus):           0.000              Jarque-Bera (JB):          839144.122
Skew:                    3.681              Prob(JB):                  0.000     
Kurtosis:                24.965             Condition No.:             48005459  
=================================================================================
Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly
specified.
[2] The condition number is large, 4.8e+07. This might indicate
that there are strong multicollinearity or other numerical
problems.
"""

In [59]:
train_processed = train_processed.drop(['Type','Features','displacement'],axis=1)
test_processed = test_processed.drop(['Type','Features','displacement'],axis=1)

In [60]:
X = train_processed.drop(columns='price')  
y = train_processed['price']  


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


xgb_reg = xgb.XGBRegressor(random_state=42)


xgb_reg.fit(X_train, y_train)

y_pred = xgb_reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

RMSE: 21835.984510314465


In [61]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import lightgbm as lgb

def evaluate_models(X, y):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the models
    models = {
        'Linear Regression': LinearRegression(),
        'Decision Tree': DecisionTreeRegressor(),
        'Random Forest': RandomForestRegressor(),
        'Gradient Boosting': GradientBoostingRegressor(),
        'AdaBoost': AdaBoostRegressor(),
        'K-Nearest Neighbors': KNeighborsRegressor(),
        'Neural Network': MLPRegressor(max_iter=1000),
        'XGBoost': xgb.XGBRegressor(),
        'LightGBM': lgb.LGBMRegressor()
    }

     # Train and evaluate each model
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        r2 = r2_score(y_test, y_pred)
        print(f'{name}: RMSE = {rmse:.4f}, R² = {r2:.4f}')

X = train_processed.drop(columns='price') 
y = train_processed['price']
evaluate_models(X, y)

Linear Regression: RMSE = 22927.2870, R² = 0.4069
Decision Tree: RMSE = 29560.6671, R² = 0.0140
Random Forest: RMSE = 22646.8143, R² = 0.4213
Gradient Boosting: RMSE = 21517.7241, R² = 0.4776
AdaBoost: RMSE = 24934.6900, R² = 0.2985
K-Nearest Neighbors: RMSE = 23333.2845, R² = 0.3857
Neural Network: RMSE = 22646.3145, R² = 0.4213
XGBoost: RMSE = 21835.9845, R² = 0.4620
LightGBM: RMSE = 21401.7424, R² = 0.4832


In [62]:
model = lgb.LGBMRegressor()
X = train_processed.drop(columns='price') 
y = train_processed['price']
model.fit(X, y)

test = test_processed.drop(['id'],axis=1)
y_pred = model.predict(test)

result_df = pd.DataFrame({'id': test_processed['id'], 'price': y_pred})

# Save the result DataFrame to a CSV file
result_df.to_csv('C:/Users/Aravind/PROJECTS/Used-Car-Price-Prediction/Predictions/transmission-nb.csv', index=False)